In [1]:
import pandas as pd
from sqlalchemy import create_engine, types
from sqlalchemy import text 
from dotenv import dotenv_values

In [2]:
config = dotenv_values()

pg_user = config['POSTGRES_USER'] 
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

In [3]:
engine = create_engine(url, echo=False)

In [4]:
with engine.begin() as conn: 
    result = conn.execute(text(f'SET search_path TO {pg_schema};'))

In [5]:
flights = pd.read_sql(sql=text("SELECT * FROM debby_prep_flights;"), con=engine)

In [6]:
flights.head()

,flight_date,dep_time,sched_dep_time,dep_delay,dep_delay_interval,arr_time,sched_arr_time,arr_delay,arr_delay_interval,airline,...,flight_number,origin,dest,air_time,air_time_interval,actual_elapsed_time,actual_elapsed_time_interval,distance_km,cancelled,diverted
0,2024-08-01,01:44:00,00:49:00,55.0,0 days 00:55:00,07:23:00,06:37:00,46.0,0 days 00:46:00,F9,...,1734,DEN,MCO,190.0,0 days 03:10:00,219.0,0 days 03:39:00,2488.05,0,0
1,2024-08-01,02:02:00,02:22:00,-20.0,-1 days +23:40:00,05:04:00,05:27:00,-23.0,-1 days +23:37:00,NK,...,883,SJU,MCO,144.0,0 days 02:24:00,182.0,0 days 03:02:00,1913.51,0,0
2,2024-08-01,06:18:00,02:54:00,204.0,0 days 03:24:00,09:04:00,05:59:00,185.0,0 days 03:05:00,F9,...,1097,BQN,MCO,146.0,0 days 02:26:00,166.0,0 days 02:46:00,1816.95,0,0
3,2024-08-01,03:59:00,03:03:00,56.0,0 days 00:56:00,06:45:00,06:09:00,36.0,0 days 00:36:00,F9,...,1823,SJU,MCO,146.0,0 days 02:26:00,166.0,0 days 02:46:00,1913.51,0,0
4,2024-08-01,04:31:00,03:30:00,61.0,0 days 01:01:00,08:42:00,07:30:00,72.0,0 days 01:12:00,B6,...,746,PSE,JFK,227.0,0 days 03:47:00,251.0,0 days 04:11:00,2602.31,0,0


In [7]:
weather = pd.read_sql(sql=text("SELECT * FROM debby_prep_weather_daily;"), con=engine)

In [8]:
weather.head(3)

,airport_code,station_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,max_snow_mm,avg_wind_direction,avg_wind_speed_kmh,wind_peakgust_kmh,avg_pressure_hpa,sun_minutes,date_day
0,TPA,72211,2024-08-01,28.9,26.1,33.3,11.2,0,55,6.5,None,1019.4,None,1.0
1,JFK,74486,2024-08-01,28.4,23.9,33.9,0.0,0,220,14.8,None,1013.6,None,1.0
2,MCO,72205,2024-08-01,28.7,25.0,35.6,0.3,0,23,7.2,None,1019.5,None,1.0


In [9]:
weather_TPA = weather.query('airport_code == "TPA"')
weather_TPA.head(3)

,airport_code,station_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,max_snow_mm,avg_wind_direction,avg_wind_speed_kmh,wind_peakgust_kmh,avg_pressure_hpa,sun_minutes,date_day
0,TPA,72211,2024-08-01,28.9,26.1,33.3,11.2,0,55,6.5,None,1019.4,None,1.0
3,TPA,72211,2024-08-02,28.3,24.4,33.9,31.2,0,0,6.1,None,1017.6,None,2.0
6,TPA,72211,2024-08-03,27.7,25.0,32.8,1.8,0,72,9.4,None,1014.4,None,3.0


In [10]:
weather_JFK = weather.query('airport_code == "JFK"')
weather_MCO = weather.query('airport_code == "MCO"')

In [11]:
weather_JFK.describe()

,station_id,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,max_snow_mm,avg_wind_direction,avg_wind_speed_kmh,avg_pressure_hpa,date_day
count,31.0,31.000000,31.000000,31.000000,31.000000,31.0,31.000000,31.000000,31.000000,31.000000
mean,74486.0,24.100000,20.212903,28.567742,3.577419,0.0,200.870968,16.003226,1016.419355,16.000000
std,0.0,2.264803,2.578726,2.910943,7.300124,0.0,89.123039,3.814576,4.603869,9.092121
min,74486.0,19.400000,14.400000,22.800000,0.000000,0.0,3.000000,12.200000,1005.700000,1.000000
25%,74486.0,22.950000,18.600000,26.950000,0.000000,0.0,131.000000,13.500000,1013.450000,8.500000
50%,74486.0,24.000000,20.600000,28.300000,0.000000,0.0,218.000000,15.100000,1015.800000,16.000000
75%,74486.0,25.600000,22.200000,30.600000,4.600000,0.0,270.000000,17.100000,1019.050000,23.500000
max,74486.0,28.400000,23.900000,35.000000,32.500000,0.0,327.000000,31.700000,1024.900000,31.000000
